In [115]:
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
import random
import math
import os

In [116]:
location_feature = pickle.load(open("pickles/location_feature.pkl","rb"))
title_overlap = pickle.load(open("pickles/title_overlap.pkl","rb"))
context_count = pickle.load(open("pickles/context_count.pkl","rb"))
cue_words = pickle.load(open("pickles/cue_words_reduced.pkl", "rb"))
tags = pickle.load(open("pickles/baseline_tags.pkl", "rb"))
contexts = pickle.load(open("pickles/reduced_context.pkl", "rb"))

In [117]:
contexts_dist = pickle.load(open("pickles/contexts_dist.pkl", "rb"))

In [118]:
print(contexts['W15-2201'][1])

{'paper_name': 'joint morphological and syntactic analysis for richly inflected languages.', 'context': ['ut', 'half', 'size', 'use', 'unlabel', 'sentenc', 'chemic', 'domain', 'share', 'ratio', 'train', 'set', 'size', 'unlabel', 'data', 'set', 'size', 'compar', 'web', 'domain', 'data', 'set', 'keep', 'scale', 'train', 'unlabel', 'set', 'allow', 'easili', 'adapt', 'best', 'set', 'web', 'domain', 'experi', '<number>', 'depend', 'parser', 'use', 'mate', 'transitionbas', 'depend', 'parser', 'default', 'set', 'experi', 'cf', '<this_citation>', 'tag', 'use', 'predict', 'po', 'tag', 'carri', 'experi', 'believ', 'realist', 'scenario', 'parser', 'intern', 'tagger', 'use', 'suppli', 'po', 'tag', 'unlabel', 'set', 'test', 'dataset', 'order', 'compar', 'previou', 'work', 'evalu', 'approach', 'addit', 'gold', 'po', 'tag', 'text', 'chemic', 'domain', 'gold', 'tag', 'use', 'previou', 'work', 'baselin', 'gener', 'train', 'p']}


In [119]:
fixed_context = pickle.load(open("pickles/fixed_context.pkl","rb"))

In [120]:
print(len(context_count.keys()))

1389


In [121]:
num_table = pickle.load(open("pickles/num_table.pkl","rb"))

In [122]:
cit_titles = pickle.load(open("pickles/citation_titles.pkl","rb"))
ids = tags.keys()
ids = list(set(ids).intersection(set(num_table.keys())))

In [123]:
print(len(ids))

1389


In [124]:
print(ids)

['K15-1005', 'P11-1062', 'N13-1123', 'N03-1026', 'W12-4205', 'D10-1083', 'W15-4911', 'D14-1032', 'P00-1039', 'P03-1032', 'W06-1641', 'D13-1179', 'P12-1069', 'P96-1006', 'D10-1004', 'D15-1004', 'W15-2138', 'E12-1051', 'D12-1037', 'D12-1042', 'D10-1079', 'W13-3512', 'S15-1002', 'N10-1128', 'P08-1106', 'P10-1103', 'E09-1057', 'P98-2159', 'E09-1005', 'P13-1078', 'P13-1107', 'P12-1060', 'P14-1070', 'N06-1006', 'N07-1026', 'D11-1138', 'P11-1054', 'N10-1085', 'J00-4001', 'D09-1159', 'D12-1069', 'D12-1085', 'P13-1042', 'S14-1003', 'D13-1134', 'N09-1050', 'J08-1003', 'W11-0319', 'E93-1007', 'J08-1001', 'S14-1002', 'J05-1003', 'D12-1064', 'D10-1075', 'D15-1015', 'W06-1606', 'Q15-1011', 'D12-1056', 'D15-1102', 'D15-1038', 'P14-1006', 'P14-1087', 'E12-1006', 'P08-1088', 'D09-1037', 'P06-1085', 'D13-1033', 'D12-1120', 'E09-1088', 'D11-1083', 'D10-1081', 'E14-1073', 'N12-1031', 'P10-1020', 'D13-1202', 'D14-1133', 'P15-1043', 'P10-1138', 'D08-1023', 'P04-1048', 'P15-1071', 'P10-1009', 'D13-1016', 'P1

In [125]:
ids_old = []
for file in os.listdir("xmls/"):
    ids_old.append(file[:8])

In [126]:
ids = list(set(ids).intersection(set(ids_old)))

In [127]:
len(ids)

1181

In [128]:
dataset = []
c = 0
for key in ids :
    papers = cit_titles[key]
    for paper in papers :
        data = {}
        data['paper_id'] = key
        data['citation_title'] = paper
        for pap in location_feature[key]:
            if(pap['paper_name']==paper) :
                data['location_feature'] = pap['location_feature']
                break
                
        for pap in num_table[key]:
            if(pap['paper_name']==paper) :
                data['num_table'] = pap['num_table']
                break
                
        for pap in title_overlap[key]:
            if(pap['paper_name']==paper) :
                data['title_overlap'] = pap['overlap']
                break
                
        for pap in context_count[key]:
            if(pap['paper_name']==paper) :
                data['context_count'] = pap['context_count']
                break
                
        for pap in contexts_dist[key] :
            if(pap['paper_name']==paper) :
#                 data['context'] = pap['context']
                data['distances'] = pap['distances']
                break
        
        for pap in contexts[key] :
            if(pap['paper_name']==paper) :
                data['context'] = pap['context']
                break
                
        for pap in cue_words[key]:
            if(pap['paper_name']==paper) :
                data['cue_count'] = pap['cue_count']
                break
                
        for pap in tags[key]:
            if(pap['paper_name']==paper) :
                if(pap['tag']==1):
                    data['label'] = 'baseline'
                else :
                    data['label'] = 'non_baseline'
                break
         
        for pap in fixed_context[key]:
            if(pap['paper_name']==paper) :
                data['fixed_context'] = pap['fixed_context']                   
        dataset.append(data)   
        
print(c)

0


In [129]:
print(len(dataset))

31943


In [130]:
final_dataset = []
for data in dataset :
    if(data['context_count']!=0):
        final_dataset.append(data)
print(len(final_dataset))

27636


In [131]:
dataset = final_dataset

In [132]:
trainset = dataset[:int(0.8*len(dataset))]
testset = dataset[int(0.8*len(dataset)):]

In [133]:
def sanity_check(probs):

	sum = 0
	for key, value in probs.items():
		sum+=value
	sum = round(sum, 7)
	return sum==1

In [134]:
def lm_model(trainset, testset):

    word_baseline_frequency = {}
    word_non_baseline_frequency = {}
    baseline_length = 0
    non_baseline_length = 0

    for i in trainset:
        # if len(i['fixed_context'])>1 and i['label']=='baseline':
        # 	continue
        for context in i['fixed_context']:
            if i['label']=='baseline':
                baseline_length+=len(context)
            else:
                non_baseline_length+=len(context)
            for word in context:
                if word not in word_baseline_frequency:
                    word_baseline_frequency[word] = 0
                    word_non_baseline_frequency[word] = 0
                if i['label']=='baseline':
                    word_baseline_frequency[word]+=1
                else:
                    word_non_baseline_frequency[word]+=1

    final_word_baseline_frequency = {}
    final_word_baseline_frequency['<unk>'] = 0
    word_non_baseline_frequency['<unk>'] = 0

    for word in word_baseline_frequency:
        if word_baseline_frequency[word]>5:	
            final_word_baseline_frequency[word] = word_baseline_frequency[word]
        else:
            final_word_baseline_frequency['<unk>'] += word_baseline_frequency[word]
            word_non_baseline_frequency['<unk>'] += word_non_baseline_frequency[word]
            del word_non_baseline_frequency[word]

    word_baseline_frequency = final_word_baseline_frequency

    word_baseline_probability = {}
    word_non_baseline_probability = {}

    for word in word_baseline_frequency:
        word_baseline_probability[word] = (word_baseline_frequency[word]+1)/(baseline_length+len(word_baseline_frequency))
        word_non_baseline_probability[word] = (word_non_baseline_frequency[word]+1)/(non_baseline_length+len(word_non_baseline_frequency))

    # print(word_baseline_probability['<unk>'])
    # print(word_non_baseline_probability['<unk>'])

    assert sanity_check(word_baseline_probability) and sanity_check(word_non_baseline_probability)

    conf_mat1 = np.zeros((2,2))
    conf_mat2 = np.zeros((2,2))

    min_p = 1

    for i in trainset+testset:
        P_b = 0
        P_nb = 0
        maxP_b = 0
        minP_nb = 1
        n_better = 0
        for context in i['fixed_context']:
            
            P_b_con = 1
            P_nb_con = 1
            for word in context:
                if word not in word_baseline_probability:
                    P_b_con*=word_baseline_probability['<unk>']
                    P_nb_con*=word_non_baseline_probability['<unk>']
                else:
                    P_b_con*=word_baseline_probability[word]
                    P_nb_con*=word_non_baseline_probability[word]
            if P_b_con>P_nb_con:
                n_better+=1
            P_b+=P_b_con
            P_nb+=P_nb_con
            maxP_b = max(P_b, P_b_con)
            minP_nb = min(P_nb, P_nb_con)
            P_b/=len(i['fixed_context'])
            P_nb/=len(i['fixed_context'])

        # if P_b>P_nb:
        # 	print('yes')

        # i['P_b'] = P_b
        # i['P_nb'] = P_nb
        min_p = min(min_p, P_b, P_nb)
        i['lmp'] = [P_b, P_nb, maxP_b, minP_nb, P_b/P_nb, maxP_b/minP_nb, n_better]

    for i in trainset:	
        # if len(i['fixed_context'])>1 and i['label']=='baseline':
        # 	testset.append(i)
        # 	continue
        i['lmp'][0]/=min_p
        i['lmp'][1]/=min_p
        i['lmp'][2]/=min_p
        i['lmp'][3]/=min_p
        if i['lmp'][0]>i['lmp'][1]:
            if i['label']=='baseline':
                conf_mat1[0, 0]+=1
            else:
                conf_mat1[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat1[1, 0]+=1
            else:
                conf_mat1[1, 1]+=1
        if i['lmp'][2]>i['lmp'][3]:
            if i['label']=='baseline':
                conf_mat2[0, 0]+=1
            else:
                conf_mat2[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat2[1, 0]+=1
            else:
                conf_mat2[1, 1]+=1

    # print(conf_mat1)
    prec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[0,1])
    rec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
    conf_mat1 = np.zeros((2,2))

    # print(conf_mat2)
    prec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[0,1])
    rec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
#     conf_mat2 = np.zeros((2,2))

    for i in testset:
        i['lmp'][0]/=min_p
        i['lmp'][1]/=min_p
        i['lmp'][2]/=min_p
        i['lmp'][3]/=min_p
        if i['lmp'][0]>i['lmp'][1]:
            if i['label']=='baseline':
                conf_mat1[0, 0]+=1
            else:
                conf_mat1[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat1[1, 0]+=1
            else:
                conf_mat1[1, 1]+=1
        if i['lmp'][2]>i['lmp'][3]:
            if i['label']=='baseline':
                conf_mat2[0, 0]+=1
            else:
                conf_mat2[0, 1]+=1
        else:
            if i['label']=='baseline':
                conf_mat2[1, 0]+=1
            else:
                conf_mat2[1, 1]+=1

    # print(conf_mat1)
    prec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[0,1])
    rec = conf_mat1[0,0]/(conf_mat1[0,0]+conf_mat1[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
    conf_mat1 = np.zeros((2,2))

    # print(conf_mat2)
    prec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[0,1])
    rec = conf_mat2[0,0]/(conf_mat2[0,0]+conf_mat2[1,0])
    print(prec, rec, 2*prec*rec/(prec+rec))
    conf_mat2 = np.zeros((2,2))

In [135]:
lm_model(trainset,testset)

0.2964485981308411 0.7232102143182855 0.4205223385920721
0.25894213506421765 0.8549931600547196 0.39749841000636
0.23736263736263735 0.5955882352941176 0.33944473546359355
0.2523804312137463 0.8425283156740957 0.38841165571837627


In [136]:
def context_feature(trainset, testset) :
    total_counts = {}
    citation_frequency = {}
    paper_frequency = {}
    curr_doc = trainset[0]['paper_id']
    paper_words = set([])

    for i in trainset:
        if i['paper_id']!=curr_doc:
            for word in paper_words:
                if word not in paper_frequency:
                    paper_frequency[word] = 0
                paper_frequency[word]+=1
            paper_words = set([])
            curr_doc = i['paper_id']
        words = set(i['context'])
        paper_words = paper_words.union(words)
        for word in words:
            if word not in total_counts:
                total_counts[word] = 0
                citation_frequency[word] = 0
            citation_frequency[word]+=1
        counts = {word:0 for word in words}
        for word in i['context']:
            counts[word]+=1
            total_counts[word]+=1

    for word in paper_words:
        if word not in paper_frequency:
            paper_frequency[word] = 0
        paper_frequency[word]+=1

    final_citation_frequency = {word:count for word, count in citation_frequency.items()}
    for i in citation_frequency:
        if paper_frequency[i]<=20:
            del total_counts[i]
            del final_citation_frequency[i]

    citation_frequency = final_citation_frequency
    
    dataset = trainset+testset
    
    idfs = {}

    for word in citation_frequency:
        idfs[word] = math.log10(len(dataset)*0.8/citation_frequency[word])
    
    total_counts['<unk>'] = 0
    
    for i in range(len(dataset)):
        words = set(dataset[i]['context']) & set(total_counts.keys())
        unigram_counts = {word:0 for word in words}
        try:	
            del unigram_counts['<SOS>']
            del unigram_counts['<EOS>']
        except:
            pass

        for j in range(len(dataset[i]['context'])):
            word = dataset[i]['context'][j]
            if word not in unigram_counts:
                continue
            else:
                unigram_counts[word]+=1-dataset[i]['distances'][j]


        dataset[i]['unigrams'] = unigram_counts


    total_unigram_counts = total_counts
    features = list(total_unigram_counts.keys())
    ngram_to_idx = {features[i]:i for i in range(len(features))}
    
    pickle.dump(total_counts, open('pickles/total_counts.pkl', 'wb'))
    pickle.dump(dataset, open('pickles/data_ngram.pkl', 'wb'))
    pickle.dump(ngram_to_idx, open('pickles/ngram_to_idx.pkl', 'wb'))
    pickle.dump(idfs, open('pickles/idfs.pkl', 'wb'))
    
    data_mat = np.zeros((len(dataset), len(ngram_to_idx)+35+1+5+1))

    for i in range(len(dataset)):
        context_counts = {}
        for word in dataset[i]['unigrams']:
            if word not in context_counts:
                context_counts[word] = 0
            context_counts[word]+=dataset[i]['unigrams'][word]
            if word in idfs:
                data_mat[i, ngram_to_idx[word]] = dataset[i]['unigrams'][word]

        dataset[i]['context_feature'] = data_mat[i, :len(ngram_to_idx)]
        
    return dataset

In [137]:
dataset = context_feature(trainset, testset)

In [147]:
values = []
output = []
for data in dataset :
    ar = []
    ar.append(data['context_count'])
    ar.append(data['title_overlap'])
    ar.append(data['cue_count'])
#     ar.append(sum(data['location_feature']))
    ar.append(data['num_table'])
    ar.extend(data['location_feature'])
    ar.extend(data['lmp'])
#     ar.extend(data['context_feature'])
    values.append(ar)
    if(data['label']=='baseline'):
        output.append(1)
    else :
        output.append(0)

In [148]:
values = np.array(values)
output = np.array(output)

In [149]:
print(values.shape)

(27636, 2744)


In [150]:
# std_scale = MinMaxScaler((0,1))
# values[:,0:-6] = std_scale.fit_transform(values[:,0:-6])
scaler = MinMaxScaler((0,1))
values = scaler.fit_transform(values)

In [151]:
def split(data, output):
    n = len(data)
    last = int(0.8*n)
    train_data = data[:last]
    train_output = output[:last]
    test_data = data[last:]
    test_output = output[last:]
    return train_data, test_data, train_output, test_output

In [152]:
train_data, test_data, train_output, test_output = split(values, output)

In [153]:
def shuffle(train_data, train_output) :
    baselines = []
    non_baselines = []
    for i in range(len(train_output)) :
        if(train_output[i]==1) :
            baselines.append(train_data[i])
        else :
            non_baselines.append(train_data[i])
    
    n = len(baselines)
    ar = np.random.choice(len(non_baselines), len(baselines))
    nb_ar = []
    for x in ar :
        nb_ar.append(non_baselines[x])
        
    data = []
    data.extend(nb_ar)
    output = []
    for i in range(len(data)):
        output.append(0)
    
    data.extend(baselines)
    for i in range(n) :
        output.append(1)
        
    total_data = []
    for i in range(len(data)):
        ar = []
        ar.append(data[i])
        ar.append(output[i])
        total_data.append(ar)
        
    total_data = np.array(total_data)
    np.random.shuffle(total_data)
    
    data = []
    output = []
    for ar in total_data :
        data.append(ar[0])
        output.append(ar[1])
        
    data = np.array(data)
    output = np.array(output)
    
    return data, output

In [154]:
train_data, train_output = shuffle(train_data, train_output)

In [155]:
params = {'C':[0.1, 0.5, 1, 2], 'fit_intercept':[True, False]}
modelin = LogisticRegression(solver='lbfgs', max_iter=1e4, n_jobs=5, random_state=1, warm_start=False, C=1)
model = GridSearchCV(modelin, params, cv=2, n_jobs=5)
#model = LogisticRegression(solver='lbfgs',max_iter=1e4, n_jobs=5, random_state=1, warm_start=False, C=1, fit_intercept=True)
clf = model.fit(train_data, train_output)
predict_test = clf.predict(test_data)
print(classification_report(test_output, predict_test))
predictions = clf.predict(train_data)
print(classification_report(train_output, predictions))
print(model.best_params_)

              precision    recall  f1-score   support

           0       0.97      0.88      0.92      4984
           1       0.40      0.73      0.51       544

    accuracy                           0.87      5528
   macro avg       0.68      0.80      0.72      5528
weighted avg       0.91      0.87      0.88      5528

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      2193
           1       0.96      0.89      0.92      2193

    accuracy                           0.92      4386
   macro avg       0.93      0.92      0.92      4386
weighted avg       0.93      0.92      0.92      4386

{'C': 2, 'fit_intercept': True}
